# Shakespeare


```{contents}
:local:
```


In [36]:
from __future__ import annotations

import os
import time
from pathlib import Path
from typing import Any, Dict, List, Literal, Tuple, Union

import numpy as np
from numpy.typing import ArrayLike, NDArray
import requests
import tiktoken
import torch
from rich.pretty import pprint
from torch import nn
from torch.utils.data import DataLoader, Dataset
import os
import random
import warnings

import numpy as np
import torch
import torch.backends.cudnn

from pydantic import BaseModel, Field, model_validator, computed_field


## Composing the Configurations

In [37]:
class Composer(BaseModel):
    seed: int = 2024
    debug: bool = False

    url: str = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    dataset_name: str = "tinyshakespeare"
    data_folder: str = Field(default="./data/tinyshakespeare", description="Path to the data folder")

    train_path: Path = Field(None, description="Path to the train file")
    valid_path: Path = Field(None, description="Path to the valid file")

    encoding_name: Literal['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base'] = "gpt2"

    batch_size: int = Field(default=64, description="Batch size")
    block_size: int = Field(default=128, description="Block size, an alias for max length/context window size.", alias="context_length")
    device_type: Literal["cpu", "cuda"] = "cpu"
    device: torch.device = Field(None, description="Device to use")


    @model_validator(mode="after")
    def set_train_valid_paths(self) -> Composer:
        self.train_path = Path(self.data_folder) / "train.txt"
        self.valid_path = Path(self.data_folder) / "valid.txt"
        return self

    @model_validator(mode="after")
    def set_device(self) -> Composer:
        self.device = torch.device(self.device_type)
        return self

    @model_validator(mode="after")
    def set_debug_fields(self) -> Composer:
        if self.debug:
            self.batch_size = 2
            self.block_size = 8
        return self

    class Config:
        extra = "forbid"
        arbitrary_types_allowed = True

composer = Composer(debug=True)
pprint(composer)


Composer(
│   seed=2024,
│   debug=True,
│   url='https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt',
│   dataset_name='tinyshakespeare',
│   data_folder='./data/tinyshakespeare',
│   train_path=PosixPath('data/tinyshakespeare/train.txt'),
│   valid_path=PosixPath('data/tinyshakespeare/valid.txt'),
│   encoding_name='gpt2',
│   batch_size=2,
│   block_size=8,
│   device_type='cpu',
│   device=device(type='cpu')
)

## Reproducibility

In [38]:
def configure_deterministic_mode() -> None:
    """
    See https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html
    and https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility
    """
    # fmt: off
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.backends.cudnn.benchmark        = False
    torch.backends.cudnn.deterministic    = True
    torch.backends.cudnn.enabled          = False

    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    # fmt: on
    warnings.warn(
        "Deterministic mode is activated. This will negatively impact performance and may cause increase in CUDA memory footprint.",
        category=UserWarning,
        stacklevel=2,
    )


def seed_all(
    seed: int = 1992,
    seed_torch: bool = True,
    set_torch_deterministic: bool = True,
) -> int:
    """
    Seed all random number generators.

    Parameters
    ----------
    seed : int
        Seed number to be used, by default 1992.
    seed_torch : bool
        Whether to seed PyTorch or not, by default True.

    Returns
    -------
    seed: int
        The seed number.
    """
    # fmt: off
    os.environ["PYTHONHASHSEED"] = str(seed)       # set PYTHONHASHSEED env var at fixed value
    np.random.default_rng(seed)                    # numpy pseudo-random generator
    random.seed(seed)                              # python's built-in pseudo-random generator

    if seed_torch:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)           # pytorch (both CPU and CUDA)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.enabled = False

        if set_torch_deterministic:
            configure_deterministic_mode()
    # fmt: on
    return seed

In [39]:
seed_all(composer.seed, seed_torch=True, set_torch_deterministic=False)

2024

## Tokenization and Vocabulary

- https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt
- https://github.com/openai/tiktoken
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

Language models don't see text like you and I, instead they see a sequence of numbers (known as tokens). Byte pair encoding (BPE) is a way of converting text into tokens. It has a couple desirable properties[^1]:

- It's reversible and lossless, so you can convert tokens back into the original text
- It works on arbitrary text, even text that is not in the tokeniser's training data
- It compresses the text: the token sequence is shorter than the bytes corresponding to the original text. On average, in practice, each token corresponds to about 4 bytes.
- It attempts to let the model see common subwords. For instance, "ing" is a common subword in English, so BPE encodings will often split "encoding" into tokens like "encod" and "ing" (instead of e.g. "enc" and "oding"). Because the model will then see the "ing" token again and again in different contexts, it helps models generalise and better understand grammar.

In [40]:
def am_i_in_jupyter() -> bool:
    try:
        from IPython import get_ipython
        if "IPKernelApp" not in get_ipython().config:
            return False
    except ImportError:
        return False
    return True

IN_JUPYTER = am_i_in_jupyter()

In [41]:
def download(url: str, dataset_name: str, dest_folder: Path | str) -> Path:
    dest_folder_path = Path(dest_folder)

    dest_folder_path.mkdir(parents=True, exist_ok=True)

    filepath = dest_folder_path / f"{dataset_name}.txt"

    response = requests.get(url, stream=True)
    corpus = response.text
    response.raise_for_status()

    with open(filepath, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    return filepath, corpus

In [42]:
filepath, corpus = download(composer.url, composer.dataset_name, composer.data_folder)
pprint(filepath)
print(corpus[:100])

PosixPath('data/tinyshakespeare/tinyshakespeare.txt')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [43]:
N = len(corpus)
train_data = corpus[: int(N * 0.9)]
valid_data = corpus[int(N * 0.9) :]

# encode with tiktoken gpt2 bpe
tokenizer = tiktoken.get_encoding(composer.encoding_name)
tiktoken.list_encoding_names()

['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base']

In [44]:
train_ids = tokenizer.encode_ordinary(train_data)
valid_ids = tokenizer.encode_ordinary(valid_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(valid_ids):,} tokens")

train has 301,966 tokens
val has 36,059 tokens


In [45]:
print(tokenizer.decode(train_ids[:2]))
print("-" * 80)
print(tokenizer.decode(train_ids[:100]))


First Citizen
--------------------------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we


In [46]:
# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
valid_ids = np.array(valid_ids, dtype=np.uint16)

train_ids.tofile(composer.train_path)
valid_ids.tofile(composer.valid_path)

# train.bin has 301,966 tokens
# val.bin has 36,059 tokens

## Dataset and Dataloading (Poor Man's Dataloader)

**TODO: to bridge the gap between the previous section and this one, we need to
explain why we need to construct a dataset.**

---

As Karpathy puts it, he implemented a poor man's
[dataloader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).
We will start by dissecting the code and understanding how it works and finally,
show that everything can be done with PyTorch's `Dataset` and `Dataloader`.

### Memory Mapping

Firstly, Karpathy uses `numpy`'s
[memory mapping](https://numpy.org/doc/stable/reference/generated/numpy.memmap.html)
(`numpy.memmap`) to load the data. Memory mapping is used to create a
memory-mapped array from a binary file. This involves mapping the contents of a
file directly into the virtual memory space of the calling process. This allows
applications to access the file data as if it were loaded in memory, using
pointer operations or array indexing, without the need for explicit read or
write operations.

This essentially means that you can access small segments of a large file
without having to load the entire file into memory. The concept draws
similarities to the use of [generators](https://wiki.python.org/moin/Generators)
in Python, where you can iterate over a large dataset without having to load the
entire dataset into memory.

In [47]:
train_data = np.memmap(composer.train_path, dtype=np.uint16, mode="r")
train_data_dtype = train_data.dtype
train_data_shape = train_data.shape

print(f"data_dtype: {train_data_dtype}, data_shape: {train_data_shape}")

data_dtype: uint16, data_shape: (301966,)


We see that the shape of train data is `(301966,)`, which means that it is a 1D (flattened) array 
with $301966$ elements - this is basically the length of the entire train corpus, in terms of
tokens.

### Notation, Context Length, Shuffling and Batching

However, we are not going to pass the entire training corpus as is to the model.
Instead, we are going to pass a **batch** of sequences (each sequence of length
`context_length`) to the model at a time.

#### Notation

Let's consider a sequence $\mathbf{x} = (x_1, x_2, \ldots, x_T)$, where:

-   $x_t$ represents the $t$-th token in the sequence,
-   Each token $x_t$ is an element of a predefined vocabulary $\mathcal{V} := \mathcal{X}$,
-   $T$ denotes the total number of tokens in the sequence, i.e., the sequence
    length.

In practice, we handle multiple sequences at once by grouping them into a batch.
This batch, denoted as $\mathcal{B}$, is then presented to the model for
parallel processing.

A batch of sequences is represented as a matrix $\mathbf{X}$, where each row
corresponds to a sequence in the batch. If the batch size is $\mathcal{B}$ and
each sequence within the batch has a fixed length $T$, then $\mathbf{X}$ can be
expressed as:

$$
    \mathbf{X} = \begin{bmatrix} x_{1,1} & x_{1,2} & \ldots & x_{1,T} \\ x_{2,1}
        & x_{2,2} & \ldots & x_{2,T} \\ \vdots & \vdots & \ddots & \vdots \\ x_{\mathcal{B},1} &
        x_{\mathcal{B},2} & \ldots & x_{\mathcal{B},T} \\\end{bmatrix} \in \mathbb{Z}^{\mathcal{B} \times T}
$$

Here, $x_{i,j}$ denotes the $j$-th token in the $i$-th sequence of the batch.
It's important to note that while we represent the sequences in a real-valued
space $\mathbb{Z}^{\mathcal{B} \times T}$ for mathematical convenience, in
practice, each $x_{i,j}$ corresponds to a discrete token from the vocabulary
$\mathcal{X}$ so using $\mathbb{Z}^{+}$ would be more appropriate.

#### Context Length / Block Size

$T$ is often referred to as the sequence length, or in the context of GPT, it is
the `block_size` or `context_length` or `max_seq_len`.

It is the length of the sequence that the model will be trained on and is also
the context length/context window that we often hear about.

For example,
[Gemini 1.5](https://blog.google/technology/ai/google-gemini-next-generation-model-february-2024)
was announced to have a standard $128,000$ token context window, up to a maximum
of $1$ million max length.

Typically, I think that if your model is trained on a certain context length, it
is not trivial to change it. For example, if you train a model on a context
length of $128$, you cannot simply change it to $256$ without retraining the
model. But it seems that it is increasingly possible to do so.

Let's look at an example, if we define our $L$ to be $32$, then we would expect each
sequence to be of length $32$.

In [48]:
first_sequence = train_data[0:0+32]
pprint(first_sequence)

first_sequence_decoded = tokenizer.decode(first_sequence)
print(first_sequence_decoded)

memmap([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,
│   │      11,  3285,   502,  2740,    13,   198,   198,  3237,    25,
│   │     198,  5248,   461,    11,  2740,    13,   198,   198,  5962,
│   │   22307,    25,   198,  1639,   389], dtype=uint16)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are


The example is just extracting $1$ such sequence $\mathbf{x}$ from the train
corpus. To leverage the prowess of linear algebra operations in CUDA, we would
typically pass a batch of sequences $\mathcal{B}$ to the model at a time.

Furthermore, we would require some level of randomness in the sequences that we
pass to the model to enable generalisation. You really do not want the model to
overfit to an ordered sequence of tokens in the training corpus.

To this end, let's see how Karpathy implements batching and shuffling of the
sequences.

#### Shuffling and Discrete Uniform Sampling

To enable shuffling, Karpathy generates a tensor of random integers (essentially a list of
random integers), which serve as indices. These indices are used to select
random sequences from the training (and validation) data.

For simplicity, let's look at the case where batch size is reduced to $\mathcal{B} = 1$.
This means we only need to sample $1$ sequence from the training data - and consequently
we only need $1$ random index.

We can easily achieve this via `torch.randint` which generates random integers
from a discrete uniform distribution over the half-open interval $[l, h)$,
and since we only want to sample $1$ sequence, we set `size=(1,)`.

In [49]:
generator = torch.Generator(device=composer.device)
low, high = 0, len(train_data) - composer.block_size
size = (1,)
indices: torch.Tensor = torch.randint(low=low, high=high, size=size, generator=generator)
pprint(indices)
pprint(indices.shape)

tensor([136016])

torch.Size([1])

The mathematical operation performed by `torch.randint(low, high, size, generator)` can be described as drawing samples from a uniform discrete distribution. Each element of the resulting tensor is an independent and identically distributed {cite}`radford2019language` (i.i.d.) random variable $X_i$ with the following probability mass function (PMF):

$$
\mathbb{P}(X_i = k) = \frac{1}{h - l} \quad \text{for} \, k = l, \ldots, h-1 
$$

This PMF implies that each integer in the range $[l, h-1]$ has an equal probability of being selected.

In our demonstration, we selected a random index, specifically $136,016$, from
our training dataset. This serves as a starting point for constructing a
sequence, denoted as $\mathbf{x}$. This sequence consists of the token found at
the chosen index and extends to include the subsequent $T$ tokens, where $T$
represents the block size. For the sake of simplicity, and to align with our
predefined settings, we have chosen $T = 8$. This block size is predetermined in
our `composer` configuration, activated specifically under a `debug` mode.

In code, we can achieve this by slicing the training data from the random index
to the random index plus the block size. This is done by `train_data[random_index:random_index+block_size]`.

In [50]:
random_sequence = train_data[indices:indices+composer.block_size]
pprint(random_sequence)
pprint(random_sequence.shape)

random_sequence_decoded = tokenizer.decode(random_sequence)
tokenizer.decode(random_sequence)

memmap([3194,  612,   11,  290,  284,  606,  910,   11], dtype=uint16)

(8,)

' written there, and to them say,'

One might wonder why the highest value of the random integers is
`len(self.train_data) - self.block_size`. This is mostly to prevent index out of
range errors. As we shall soon see, we are using these `indices` to slice a
sequence of length `block_size` from the data where you start slicing from the
index `index` and end at `index + block_size`.

#### Batching

Now that we understand how to sample a single sequence from the training data,
let's look at how we can sample a batch of sequences.
PyTorch made it easy for you, as we can just simply change the `size` parameter
to `(batch_size,)` so we can sample $\mathcal{B}$ number of indices - and
consequently $\mathcal{B}$ number of sequences.

In our case, if we set $\mathcal{B} = 2$, we would expect to get $2$ random
indices.

In [51]:
generator = torch.Generator(device=composer.device)
low, high = 0, len(train_data) - composer.block_size
size = (composer.batch_size,)
indices: torch.Tensor = torch.randint(low=low, high=high, size=size, generator=generator)
pprint(indices)
pprint(indices.shape)

tensor([136016, 197976])

torch.Size([2])

We then construct a batch of 
input sequences $\mathcal{B}$ by selecting the tokens at the
indices $136,016$ and $197,976$ and the next $T$ tokens via a for loop - and using `torch.stack`
to stack the sequences into a tensor of shape $\mathbb{Z}^{\mathcal{B} \times L}$.

In [52]:
x = torch.stack([torch.from_numpy((train_data[index : index + composer.block_size]).astype(np.int64)) for index in indices])
pprint(x)
pprint(x.shape)

tensor([[ 3194,   612,    11,   290,   284,   606,   910,    11],
│   │   [ 4713,  5127,    13,   198, 11486,    11,   304,   260]])

torch.Size([2, 8])

It is worth reconciling the fact that the slicing uses `[index:index + block_size]` and
therefore completes the reasoning behind the `len(self.train_data) - self.block_size` in
the `torch.randint` function call - to prevent index out of range errors. Consider
that if we do not subtract `block_size` from the length of the training data, we might
end up with an index that is the last index of the training data, and when we add
`block_size` to it, we would end up with an index that is out of range.

### Construction of Input and Target Sequences

As we will define more formally later, GPT model is an autoregressive
self-supervised learning model{cite}`math11112451` that directly learns a
conditional probability distribution $\mathbb{P}(x_t | x_{<t} ; \Theta)$ over
the vocabulary $\mathcal{V}$ of tokens, which is conditioned on the entire
history of tokens $x_{<t} = (x_1, x_2, \ldots, x_{t-1})$.

We have seen earlier how to construct an input sequence $\mathbf{x}$ from the
training data. To put things into perspective, we consider again the first
sequence that we constructed from the training data:

$$
\mathbf{x} = \begin{bmatrix} 3194 & 612 & 11 & 290 & 284 & 606 & 910 & 11 \end{bmatrix}
$$

representing the sentence `' written there, and to them say,'`.

Given the autoregressive and self-supervised nature, in order to construct the
target sequence $\mathbf{y}$, we simply shift the input sequence by one token to
the left. This means that the target sequence $\mathbf{y}$ is:

$$
\mathbf{y} = \begin{bmatrix} 612 & 11 & 290 & 284 & 606 & 910 & 11 & 198 \end{bmatrix}
$$

representing the sentence `' there, and to them say,\n'`.

This behaviour is autoregressive because we are using the context tokens
$x_{<t}$ to predict the next token $x_t$, and self-supervised because we are
using the input sequence $\mathbf{x}$ to construct the target sequence
$\mathbf{y}$ without any external labels.

To illustrate further, the prediction process during training is cumulative:

-   For predicting $x_2$, the model uses $x_1$ as context:
    $\mathbb{P}\left(x_2 \mid x_1\right)$.
-   For predicting $x_3$, the model uses both $x_1$ and $x_2$ as context:
    $\mathbb{P}\left(x_3 \mid x_1, x_2\right)$.
-   This pattern continues, such that for predicting $x_t$, the model uses
    $x_1, x_2, \ldots, x_{t-1}$ as context:
    $\mathbb{P}\left(x_t \mid x_1, x_2, \ldots, x_{t-1}\right)$

In code, we can achieve this by simply slicing the adding a `1` to the `index`
in the `train_data` slicing operation.

In [53]:
y = torch.stack([torch.from_numpy((train_data[index + 1: index + 1 + composer.block_size]).astype(np.int64)) for index in indices])
pprint(y)
pprint(y.shape)

tokenizer.decode(y[0].numpy())

tensor([[  612,    11,   290,   284,   606,   910,    11,   198],
│   │   [ 5127,    13,   198, 11486,    11,   304,   260, 14210]])

torch.Size([2, 8])

' there, and to them say,\n'

### Asynchronous Data Loading and Prefetching

As we approach the last part of the code, Karpathy moves `x` and `y` to the
device and returns them.

In [54]:
if composer.device_type == "cuda":
    # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
    x, y = x.pin_memory().to(composer.device, non_blocking=True), y.pin_memory().to(composer.device, non_blocking=True)
else:
    x, y = x.to(composer.device), y.to(composer.device)

This is a common operation in PyTorch, where we move the data to the underlying
device (CPU or GPU or MPS) to leverage the processing capabilities of the
device. It goes without saying that modern deep learning models are trained on
GPUs - and CUDA is the de facto standard for GPU-accelerated computing.

CUDA allows a `pin_memory` and `non_blocking` parameter to be set when transferring
tensor data from CPU to GPU. The `pin_memory` parameter is used to allow `.to("cuda")`
to be more [performant](https://devblogs.nvidia.com/how-optimize-data-transfers-cuda-cc/)
as it avoids some implicit CPU-to-CPU copies. Tensors which are pinned in memory
also allow the transfer from CPU to GPU to be done asynchronously via `non_blocking` with respect to
the host[^2].

It can be useful because we can do some other work in CPU while the data is being
transferred to GPU. Consider the below scenario:

- `tensor.pin_memory().to("cuda", non_blocking=True)` will transfer the tensor
  to the GPU asynchronously, and the CPU can continue doing some other work.
- While waiting, CPU can do some other operations without waiting for the
  transfer to complete,
- Once `tensor` is transferred to the GPU, then we can do some other operations
  on the GPU.

What is worth noting is that CUDA manages the synchronization such that
operations on the GPU will not start until the transfer is complete. However, CUDA
programming is complex and is out of the scope of this post. Interested readers
can see the reference section.

### Collating Everything Together

In [64]:
def get_batch(
    composer: Composer,
    *,
    split: Literal["train", "valid"],
    batch_size: int,
    block_size: int,
    generator: torch.Generator,
    device: torch.device,
    device_type: Literal["cpu", "cuda"] = "cpu",
) -> Tuple[torch.Tensor, torch.Tensor]:
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == "train":
        data = np.memmap(composer.train_path, dtype=np.uint16, mode="r")
    else:
        data = np.memmap(composer.valid_path, dtype=np.uint16, mode="r")

    low, high = 0, len(data) - block_size
    size = (batch_size,)

    indices = torch.randint(low=low, high=high, size=size, generator=generator)

    x = torch.stack([torch.from_numpy((data[index : index + block_size]).astype(np.int64)) for index in indices])
    y = torch.stack(
        [torch.from_numpy((data[index + 1 : index + 1 + block_size]).astype(np.int64)) for index in indices]
    )
    if device_type == "cuda":
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

generator = torch.Generator(device=composer.device)
train_batch = get_batch(composer, split="train", batch_size=composer.batch_size, block_size=composer.block_size, device=composer.device, generator=generator)
x, y = train_batch
pprint(x)
pprint(y)

tensor([[ 3194,   612,    11,   290,   284,   606,   910,    11],
│   │   [ 4713,  5127,    13,   198, 11486,    11,   304,   260]])

tensor([[  612,    11,   290,   284,   606,   910,    11,   198],
│   │   [ 5127,    13,   198, 11486,    11,   304,   260, 14210]])

### Using PyTorch's Dataset and Dataloader

It is relatively simple to understand - and since there is not a need to
[collate](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) the
data, which makes things a bit easier.

1. Conditional and Joint probability
2. Model definition
3. Loss see page 5/6
4. autoregressive self supervised def page 6

### Conditional Probability

The GPT model, being autoregressive, directly models the **conditional
probability** of each token given all previous tokens in a sequence. This is the
core operational mechanism of the model, allowing it to predict the next token
based on the preceding context. The conditional probability for each token $x_t$
given its predecessors $x_{<t}$ is expressed as $P(x_t | x_{<t};
\Theta)$, where
$\Theta$ represents the model's parameters.

### Joint Probability

The **joint probability** of a sequence $\mathbf{x} = (x_1, x_2, ..., x_T)$
under the model is derived from the product of these conditional probabilities.
By assuming that the probability of each token can be modeled based on its
predecessors, the model implicitly defines a joint probability distribution over
sequences. This joint probability is represented by the product:

$$
P(\mathbf{x}; \Theta) = \prod_{t=1}^{T} P(x_t | x_{<t}; \Theta)
$$

This formulation shows that while the GPT model operates through conditional
probabilities, the product of these conditional probabilities across a sequence
results in the **joint probability** of the entire sequence. Thus, the GPT model
effectively learns the joint probability distribution over sequences in the
dataset by learning to predict the next token given its context, leveraging the
chain rule of probability.


{cite}`math11112451`

- https://devblogs.nvidia.com/how-optimize-data-transfers-cuda-cc/
- https://developer.nvidia.com/blog/how-overlap-data-transfers-cuda-cc/

[^1]: [OpenAI tiktoken](https://github.com/openai/tiktoken)
[^2]: [How to Optimize Data Transfers in CUDA C/C++](https://devblogs.nvidia.com/how-optimize-data-transfers-cuda-cc/)